In [14]:
import pandas as pd
import json

<h2> ** Este proceso de ETL es similar al hecho previamente para las consultas desde la API, sin embargo, este se enfoca en optimizar el dataset para el sistema de recomendación **

In [15]:
movies = pd.read_csv('../movies_dataset.csv')

C:\Users\Usuario\AppData\Local\Temp\ipykernel_32208\307565225.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('../movies_dataset.csv')


In [18]:
#No se puede convertir la columna 'budget' a tipo numérico porque hay registros que tienen todos sus valores incorrectos, así que los eliminaremos
movies[movies['budget']== '/ff9qCepilowshEtG2GYWwzt2bs4.jpg']

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
movies[movies['budget']== '/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg']

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
movies[movies['budget']== '/zaSf5OG7V8X8gqFvly88zDdRm46.jpg']

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
movies = movies.drop(19730)

In [22]:
movies = movies.drop(29503)

In [23]:
movies= movies.drop(35587)

In [24]:
#Sin esos registros erróneos, se puede cambiar el tipo a numérico
movies['budget'] = movies['budget'].astype('float64')

Calculo el 'return' a partir de (revenue / budget). Si alguno de estos valores es 0, 'return' será también 0

In [25]:
#Defino una función que primero comprobará si alguno de los valores es 0, y si no, retornará el cálculo
def calculo_return(x, y):
    if (x == 0).any() or (y == 0).any():
        return 0
    else:
        return x / y

In [26]:
#Aplico la función anterior al dataframe, retornando los valores calculados en la columna 'return'
movies['return'] = movies.apply(lambda x: calculo_return(pd.Series(x['revenue']), pd.Series(x['budget'])), axis=1)

<h3> De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [27]:
#Reemplazo los guiones por barras para poder hacer la conversión
movies['release_date'] = movies['release_date'].str.replace("-", '/')

In [28]:
#Convierto a formato fecha y le doy el formato que se solicita
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')

In [29]:
#Extraigo el año de la fecha ya convertida
movies['release_year'] = movies['release_date'].dt.year

In [30]:
#Cambio el tipo de dato del año a int, previamente reemplazando los valores faltantes por 0
movies.fillna({'release_year':0}, inplace=True)
movies['release_year'] = movies['release_year'].astype('int')

In [31]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45463 entries, 0 to 45465
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   adult                  45463 non-null  object        
 1   belongs_to_collection  4491 non-null   object        
 2   budget                 45463 non-null  float64       
 3   genres                 45463 non-null  object        
 4   homepage               7779 non-null   object        
 5   id                     45463 non-null  object        
 6   imdb_id                45446 non-null  object        
 7   original_language      45452 non-null  object        
 8   original_title         45463 non-null  object        
 9   overview               44509 non-null  object        
 10  popularity             45460 non-null  object        
 11  poster_path            45077 non-null  object        
 12  production_companies   45460 non-null  object        
 13  p

In [32]:
#Antes de empezar a desanidar las columnas, hago transformaciones de tipos de datos a las que lo necesitan
movies['popularity'] = movies['popularity'].astype('float64')

<h3> Los valores nulos del campo release date deben eliminarse.

In [34]:
movies.drop(movies[movies['release_date'].isnull()].index, inplace=True)

<h3> Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [35]:
#importo la librería necesaria
import ast

In [36]:
#Defino una función que permitirá extraer los valores que se encuentran anidados y colocará dentro de una lista solo los valores de la clave 'name', ignorando los ID's de los géneros, países, etc.
def aplanar(columna):
    L = []
    for i in ast.literal_eval(columna):
        L.append(i['name'])
    return L

In [37]:
movies['genres'] = movies['genres'].apply(aplanar)
movies['production_companies'] = movies['production_companies'].apply(aplanar)
movies['production_countries'] = movies['production_countries'].apply(aplanar)
movies['spoken_languages'] = movies['spoken_languages'].apply(aplanar)


In [38]:
#Reemplazo las comillas simples para poder leerlo con json.loads
movies['belongs_to_collection'] = movies['belongs_to_collection'].str.replace("'", '"')

In [39]:
#Defino la función para retornar un diccionario vacío en caso de error y crear un dataframe a partir de los valores del diccionario
def excepcion_json(columna):
    try:
        return json.loads(columna)
    except:
        return {}

movies['belongs_to_collection'] = movies['belongs_to_collection'].fillna('{}')
movies_belongs = pd.json_normalize(movies['belongs_to_collection'].apply(excepcion_json))


In [40]:
#Convierto a DataFrame, voy a quedarme únicamente con el nombre de la colección, no necesito los paths de las imágenes ni los IDs
movies_belongs = pd.DataFrame(movies_belongs)

In [41]:
movies['belongs_to_collection'] = movies_belongs['name']

In [42]:
#Aplano las listas que quedaron en estas columnas
movies['genres'] = movies['genres'].apply(lambda x: ', '.join(map(str, x)))
movies['production_companies'] = movies['production_companies'].apply(lambda x: ', '.join(map(str, x)))
movies['production_countries'] = movies['production_countries'].apply(lambda x: ', '.join(map(str, x)))
movies['spoken_languages'] = movies['spoken_languages'].apply(lambda x: ', '.join(map(str, x)))

In [43]:
#Exporto a .csv el archivo con todas las transformaciones
movies.to_csv("movies_ML.csv", index=False)